In [1]:
import pandas as pd

In [108]:
df = pd.read_csv('data/list.csv')
df.head()

df["volume"] = df["x"] * df["y"] * df["z"]

#min and max
print(df['volume'].min()/1000)
print(df['x'].min())
print(df['y'].min())
print(df['z'].mean())
print(df['Waga (kg)'].max())

6000.0
300
200
233.07692307692307
40


In [152]:
class Cuboid:
    width = 0
    depth = 0
    height = 0

    def __init__(self, width, depth, height):
        self.width = width
        self.depth = depth
        self.height = height


class Package(Cuboid):
    location = None

    def __init__(self, df_package, location):
        print(df_package, location)
        super().__init__(self, df_package['x'], df_package['y'], df_package['z'])

    def addLocation(self, location):
        self.location = location


class Cursor:
    x = 0
    y = 0
    z = 0

    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def copy(self):
        return Cursor(self.x, self.y, self.z)


class Pallet(Cuboid):
    packages = []

    def __init__(self):
        super().__init__(800, 1200, 2000 - 144)

    def addPackage(self, df_package, location):
        package = Package(df_package, location)
        package.addLocation(location)
        self.packages.append(package)


def loadPackageTypes():
    packageTypes = pd.read_csv("data/listaProduktow.csv")
    return packageTypes


def loadOrder(orderNo):
    order = pd.read_csv(f"data/Zamowienia/Zamowienie{orderNo}.csv")
    return order

In [153]:


def loadPackageTypes():
    packageTypes = pd.read_csv("data/listaProduktow.csv")
    packageTypes.set_index("ID Produktu", inplace=True)
    packageTypes[['x', 'y','z']] = packageTypes['Wymiary (mm)'].str.split('x', n=2, expand=True)
    packageTypes['x'] = packageTypes['x'].astype(int)
    packageTypes['y'] = packageTypes['y'].astype(int)
    packageTypes['z'] = packageTypes['z'].astype(int)

    packageTypes.drop(columns=['Wymiary (mm)'], inplace=True)


    return packageTypes

def loadOrder(orderNo):
    order = pd.read_csv(f"data/Zamowienia/Zamowienie{orderNo}.csv")
    order.rename(columns={'Ilość': 'count'}, inplace=True)
    return order



In [154]:
def mnoz(tuples):
    wynik = 1
    for tuple in tuples:
        x, y, z = tuple
        wynik = wynik * z
    return wynik


def martyna(x,y):
    return [400, 400, 400]


def fillRow(pallet, package_row, cursor, rowSize):
    for package_depth in package_row:
        pallet.addPackage(package_depth, cursor, (rowSize[0], package_depth, rowSize[1]))
        cursor.y += package_depth


def fillLayer(pallet, package_layer, cursor, height):
    for width, package_row in package_layer:
        fillRow(pallet, package_row, cursor.copy(), (height, width))
        cursor.x += width


def package_order(raw_row, packages):
    pass



pallet = Pallet()
types = loadPackageTypes()

sumaSum = 0
for i in range(1, 21):
    df = loadOrder(i)
    df = df.join(types, on="ID Produktu", how="left", rsuffix="_types")
    size_df = df.drop(
        columns=[
            "Nazwa Produktu_types",
            # "Nazwa Produktu",
            # "Lokacja",
            # "Waga (kg)",
            # "ID Produktu",
        ]
    )
    heights = df["z"].unique()
    cursor = Cursor(0, 0, 0)
    for height in heights:
        flat_df = size_df[size_df["z"] == height]
        widths = flat_df["x"].unique()
        for width in widths:
            curr_df = flat_df[flat_df["x"] == width]

            grouped_df = curr_df.groupby('y')['count'].sum()
            tuples = [(y, count) for y, count in grouped_df.items()]
            row_packages = martyna(tuples,pallet.width)
            ordered_df = curr_df.sort_values(by=['Waga (kg)'], ascending=False)
            for depth in row_packages:
                package = ordered_df[ordered_df['y'] == depth].iloc[0]
                ordered_df[ordered_df['y'] == depth]['count'] -= 1
                #print(ordered_df[df['y'] == depth])

                pallet.addPackage(package, cursor)
                cursor.y += depth

            #print(row_packages)


            a = 3/0
            cursor.x += width


        cursor.z += height
        print(tuples, width, height)

# [
#     [
#         (600,400), (400,600)
#     ]
# ]

ID Produktu                        141
Nazwa Produktu    Walking Cheeseburger
Lokacja                        H407A08
count                               12
Waga (kg)                           20
x                                  600
y                                  400
z                                  300
Name: 0, dtype: object <__main__.Cursor object at 0x0000020443AEE350>


TypeError: Cuboid.__init__() takes 4 positional arguments but 5 were given

In [34]:
df = df.groupby('z')[['x', 'y']].apply(lambda x: ', '.join(x.astype(str))).reset_index()

In [106]:
pallet = Pallet()
cursor = Cursor(0, 0, 0)
fillLayer(pallet, layer_order, cursor.copy(), 300)
print(pallet.packages)

[200, 500, 300]


TypeError: Pallet.addPackage() takes 3 positional arguments but 4 were given